# Retrieval-Augmented Generation (RAG) Chatbot using Meta LLMs and Amazon S3

This notebook demonstrates how to build a simple Retrieval-Augmented Generation (RAG) chatbot using Meta's LLM via Amazon Bedrock. Source documents are stored in Amazon S3, and Amazon Titan is used to generate vector embeddings. The flow is orchestrated using LangChain, which handles document loading, embedding, and semantic retrieval.

## Setup

Install required libraries.

In [1]:
!pip install boto3 langchain faiss-cpu s3fs pymupdf --quiet

Import dependencies.

In [2]:
import boto3
import json
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_aws import BedrockEmbeddings
from langchain_aws import ChatBedrock

Setup constants 

In [3]:
AWS_REGION = "us-east-1"
S3_BUCKET = "lior-llama3-rag-chatbot-data-20255"
EMBED_MODEL_ID = "amazon.titan-embed-text-v1"
LLM_MODEL_ID = "meta.llama3-70b-instruct-v1:0"
PDF_FILE = 'media/sample.pdf'
PDF_S3_KEY = 'media/sample.pdf'

### Setup S3

As a demostration, we upload the local PDF to S3 in order to demostrated a full S3 integration.

In [4]:
from s3_utils import upload_file_to_s3

upload_file_to_s3(
    pdf_file=PDF_FILE,
    bucket=S3_BUCKET,
    key=PDF_S3_KEY,
    region=AWS_REGION
)


Bucket 'lior-llama3-rag-chatbot-data-20255' exists.
Uploaded media/sample.pdf to s3://lior-llama3-rag-chatbot-data-20255/media/sample.pdf


## Load Documents from Amazon S3

In [5]:
from langchain.schema import Document
import fitz

s3 = boto3.client("s3", region_name=AWS_REGION)
s3.download_file(S3_BUCKET, PDF_S3_KEY, PDF_FILE)
print(f"Downloaded PDF from S3 to {PDF_FILE}")

doc_text = ""
with fitz.open(PDF_FILE) as doc:
    for page in doc:
        doc_text += page.get_text()

# --- Wrap into LangChain Document
documents = [Document(page_content=doc_text, metadata={"source": "sample.pdf"})]

Downloaded PDF from S3 to media/sample.pdf


## Chunk and Embed Documents

Split documents into manageable chunks and embed them using Amazon Titan Embeddings.

In [13]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = splitter.split_documents(documents)

embedding = BedrockEmbeddings(model_id=EMBED_MODEL_ID)
vectordb = FAISS.from_documents(texts, embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Build Meta-Powered RAG Chain

We use Meta's Llama 3 model via Bedrock to power the chatbot with retrieval-augmented context.

In [15]:
llm = ChatBedrock(model_id=LLM_MODEL_ID)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff"
)

## Chat
Ask a question about the document that was embbeded, see the answer and the chunks that were used in order to generate it.

In [16]:
query = "Who is Alexander Adell?"
result = qa_chain({"query": query})

# Print the answer
print("\nAnswer:")
print(result["result"])

# Print the retrieved chunks used for the answer
print("\nRetrieved Chunks:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content.strip())


Answer:
Alexander Adell is a character in a science fiction story. He is one of the two main characters, along with Lupov, and is involved in the maintenance of a giant computer called Multivac.

Retrieved Chunks:

--- Chunk 1 ---
of each other and the bottle. 
 
"It's amazing when you think of it," said Adell. His broad face had lines of weariness in it, and he stirred 
his drink slowly with a glass rod, watching the cubes of ice slur clumsily about. "All the energy we can 
possibly ever use for free. Enough energy, if we wanted to draw on it, to melt all Earth into a big drop of 
impure liquid iron, and still never miss the energy so used. All the energy we could ever use, forever and 
forever and forever."

--- Chunk 2 ---
that giant computer. They had at least a vague notion of the general plan of relays and circuits that had 
long since grown past the point where any single human could possibly have a firm grasp of the whole. 
 
Multivac was self-adjusting and self-correcting. It